In [ ]:
"""
The purpose of this notebook is to cluster the data. I decided on using KModes to cluster which looks at the mode (most frequently occuring value) and
the dissimilarity of the data. The lesser the dissimlarities, the more similar the data is to each other. KModes is specifically designed to be used 
on categorical and binary data which sets it apart from other clustering algorithms like KMeans which assume that the data is continous and calculates
the euclidean distance between points. Another potential clustering algorithm that could be used is HDBScan or DBScan, however, a significant amount of 
tuning would be necessary to get accurate results. 
"""

In [ ]:
import pandas as pd
from kmodes.kmodes import KModes
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import pickle

In [ ]:
df = pd.read_csv("../data/processed_data.csv")

In [ ]:
def val_count(data):
    counter = 0
    for i in data.columns:
        print(counter)
        print(data[i].value_counts())
        counter += 1
        print("=========================="*5)
    print(data.shape)

In [ ]:
val_count(df)

In [ ]:
"""
Use this cell to find optimal k values (clusters) for KMODES with an elbow curve. Look for the elbow, where the graph sharply bends. 
Cost is a measure of how well the clustering fits the data. A lower costs means that the data is closer to the centroid, indicating 
a better fit, while a higher cost is a worse fit because the data is farther from the clusters. Optimal K values is 8.
"""

# print(df.shape)
# cost = []
# K = range(1,10)
# for k in list(K):
#     kmode = KModes(n_clusters=k, init = "Huang", n_init = 4, verbose=1) #using init = huang because the initialization is random and is computationally faster while achieving the same/better clustering results than Cao
#     kmode.fit_predict(df)
#     cost.append(kmode.cost_)
    
# plt.plot(K, cost, 'o-')
# plt.xlabel('No. of clusters')
# plt.ylabel('Cost')
# plt.title('Elbow Curve')
# plt.show()

In [ ]:
"""
Use this cell to find optimal k value for KMODES with a Silhouette Score curve. More accurate than elbow graph, but significatly more time consuming.
A Sillhouette score is a measure of how well a data point fits within the assigned cluster, a higher score means that the data is closely clustered together,
a score near 0 indicates that points overlap or on the boundaries of other clusters, and a negative score indicates that points are in completely wrong 
clusters. To read the graph, choose the cluster with the highest Sillhouette score. Optimial K values is 5.
"""

# print(df.shape)
# results = {}
# K = range(2,10)
# for k in list(K):
#     kmode = KModes(n_clusters=k, init = "Huang", n_init = 4, verbose=1) #Make sure to change the n_init argument as needed to speedup compile time
#     clusters = kmode.fit_predict(df)
#     db_index = silhouette_score(df, clusters, metric= "hamming") #MUST use hamming metric because data is binary
#     results[k] = db_index
    

# plt.plot(list(results.keys()), list(results.values()))
# plt.xlabel("Number of clusters")
# plt.ylabel("Silhouette Score")
# plt.show()

In [ ]:
# clusters the data. Use the k amount of clusters found in either the elbow curve or Sillhouette Score curve for n_clusters.

km = KModes(n_clusters= 8, init = "Huang" , n_init= 20).fit(df)
df["cluster"] = km.labels_
val_count(df)


In [ ]:
"""
This is to measure how well the clustering went. Any score of 0.5 or higher is generally good and indicates that the data is more closely associated with
its own cluster than other clusters. Scores close to 1 indciate clear seperations, scores near 0 mean clustering is more uncertain, and scores near -1 
indicate that there has likely been misclassifcation.
"""

print(f"Silhouette Score: {silhouette_score(df.drop(columns= "cluster"), df["cluster"], metric= "hamming")}")


In [ ]:
df.to_csv("../data/clustered_data.csv", index= False)

In [ ]:
with open('../models/kmodes_model.pkl', 'wb') as f:
    pickle.dump(km, f)
